In [2]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


In [6]:
df_municipios = pd.read_csv('../apis/datos/sugerencias_lugares_2024.csv')

In [11]:
municipios = df_municipios.sample(5)

In [12]:
municipios.drop(columns=['Unnamed: 0'], inplace=True)

In [13]:
municipios = municipios[['nombre_Municipio']]

In [14]:
lista_municipios = [m[0] for m in municipios.values]

lista_municipios

['Campo Real',
 'Fuente el Saz de Jarama',
 'Madrid',
 'Torrelaguna',
 'Las Rozas de Madrid']

In [16]:
lista_municipios_unidos = ["".join(m[0].split()).lower() for m in municipios.values]

lista_municipios_unidos

['camporeal',
 'fuenteelsazdejarama',
 'madrid',
 'torrelaguna',
 'lasrozasdemadrid']

## Municipio de ejemplo

In [17]:
df_codigos_municipios = pd.DataFrame({'munipio': lista_municipios})

In [18]:
df_codigos_municipios

,munipio
0,Campo Real
1,Fuente el Saz de Jarama
2,Madrid
3,Torrelaguna
4,Las Rozas de Madrid


In [20]:
lista_codigos = []
for muni in lista_municipios_unidos:
    chrome_options = Options()
    chrome_options.add_argument("--incognito")

    driver = webdriver.Chrome(options=chrome_options)
    url_wunder = f"https://www.wunderground.com/weather/es/{muni}"
    driver.get(url_wunder)


    driver.maximize_window()


    iframe_cookies = WebDriverWait(driver, 10).until(EC.presence_of_element_located(('xpath', '//*[@id="sp_message_iframe_1165301"]')))
    driver.switch_to.frame(iframe_cookies)
    try:
        sleep(1)
        print("intento aceptar las cookies")
        driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button").click()
    except:
        print("Element not found.")


    driver.switch_to.default_content()
    sleep(3)

    driver.find_element('css selector', '#inner-content > div.region-content-top > lib-city-header > div:nth-child(1) > div > div > a.station-name').click()

    sleep(2)
    texto = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located(('xpath', '//*[@id="inner-content"]/div[1]/app-dashboard-header/div[2]/div/div[2]/h1')))

    codigo = texto[0].get_attribute('innerHTML').split(' - ')[1]

    lista_codigos.append(codigo)

    driver.quit()

intento aceptar las cookies
intento aceptar las cookies
intento aceptar las cookies
intento aceptar las cookies
intento aceptar las cookies


In [21]:
lista_codigos

['ISEVIL93', 'ISEVIL93', 'IMCHUECA17', 'ITORRE406', 'ISESANLA2']

In [22]:
df_codigos_municipios['codigos'] = pd.Series(lista_codigos)

In [23]:
df_codigos_municipios

,munipio,codigos
0,Campo Real,ISEVIL93
1,Fuente el Saz de Jarama,ISEVIL93
2,Madrid,IMCHUECA17
3,Torrelaguna,ITORRE406
4,Las Rozas de Madrid,ISESANLA2


In [24]:
#df_codigos_municipios.to_csv('datos/codigos_municipios.csv')

In [25]:
df_codigos_municipios = pd.read_csv('datos/codigos_municipios.csv')

In [29]:
estaciones_meteorologicas = df_codigos_municipios['codigos'].values
estaciones_meteorologicas

array(['ISEVIL93', 'ISEVIL93', 'IMCHUECA17', 'ITORRE406', 'ISESANLA2'],
      dtype=object)

In [30]:
municipios_seleccionados = df_codigos_municipios['munipio'].values
municipios_seleccionados

array(['Campo Real', 'Fuente el Saz de Jarama', 'Madrid', 'Torrelaguna',
       'Las Rozas de Madrid'], dtype=object)

In [31]:
df = pd.DataFrame()

fechas = ["2024-1-1/2024-1-1","2024-2-1/2024-2-1","2024-3-1/2024-3-1","2024-4-1/2024-4-1","2024-5-1/2024-5-1","2024-6-1/2024-6-1","2024-7-1/2024-7-1","2024-8-1/2024-8-1","2024-9-1/2024-9-1","2024-10-1/2024-10-1"]

for e in range(0,len(estaciones_meteorologicas)):
    estacion_meteorologica = estaciones_meteorologicas[e]
    df_temp = pd.DataFrame()
    for d in range(0,len(fechas)):
        driver = webdriver.Chrome() #Para Abrir chrome solo ejecutar once
        url_wunder = f"https://www.wunderground.com/dashboard/pws/{estacion_meteorologica}/table/{fechas[d]}/monthly"
        driver.get(url_wunder)
        #Maximizar ventana
        driver.maximize_window() 
        print("intentamos seleccionar Cookies")
        select_cookies = WebDriverWait(driver,40).until(EC.presence_of_element_located(('xpath','//*[@id="sp_message_iframe_1165301"]')))
        driver.switch_to.frame(select_cookies)
        try:
            aceptar_cookies = driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button")
            aceptar_cookies.click()
            print("Encontrado")
        except:
            print("AY MI CUQUI, no la encuentro")

        driver.switch_to.default_content()
        sleep(5)

        tabla = driver.find_element("css selector","#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div > div")
        tabla_text = tabla.text
        tabla_split = tabla_text.split("\n")
        tabla_df = pd.DataFrame(tabla_split)
        df_temp = pd.concat([df_temp,tabla_df],axis=0)
        driver.quit()
        sleep(6)
    df_temp["Municipio"] = municipios_seleccionados[e]
    df_temp = df_temp.reindex(columns=["Municipio",0])
    df = pd.concat([df,df_temp],axis=0)

intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Encontrado
intentamos seleccionar Cookies
Enc

In [33]:
df.to_csv('datos/df.csv')

In [34]:
df = pd.read_csv('datos/df.csv')
df.head(3)

,Unnamed: 0,Municipio,0
0,0,Campo Real,Temperature Dew Point Humidity Speed Pressure ...
1,1,Campo Real,Date High Avg Low High Avg Low High Avg Low Hi...
2,2,Campo Real,1/1/2024 67.6 °F 53.8 °F 48.0 °F 52.3 °F 49.5 ...


In [35]:
df.drop(columns=['Unnamed: 0', 'Municipio'], inplace=True)

In [36]:
df_split = df['0'].str.split(expand=True)
df_split
df_munis = df
df_munis = df.drop(columns=['0'])
df_concat = pd.concat([df_munis,df_split],axis=1)
df_concat
df = df_concat
df = df.reset_index(drop=True)
indices_fuera = [0, 32, 62, 94, 125, 157, 188, 220, 252, 283, 300, 332, 362, 394, 425, 457, 488, 520, 
552, 583, 600, 632, 662, 694, 725, 757, 788, 820, 852, 883, 900, 932, 962, 994, 
1025, 1057, 1088, 1120, 1152, 1183, 1200, 1232, 1262, 1294, 1325, 1357, 1388, 1420, 
1452, 1483]

df_selected = df.loc[indices_fuera]
df_selected = df
df_selected2 = df_selected.drop(index=indices_fuera)
df = df_selected2.reset_index(drop=True)
df

C:\Users\JLFA\AppData\Local\Temp\ipykernel_19796\3894192525.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_concat = pd.concat([df_munis,df_split],axis=1)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,Date,High,Avg,Low,High,Avg,Low,High,Avg,Low,...,None,None,None,None,None,None,None,None,None,None
1,1/1/2024,67.6,°F,53.8,°F,48.0,°F,52.3,°F,49.5,...,0.2,mph,0.0,mph,30.03,in,29.91,in,0.00,in
2,1/2/2024,68.1,°F,53.9,°F,44.7,°F,53.7,°F,48.5,...,0.0,mph,0.0,mph,30.06,in,29.97,in,0.00,in
3,1/3/2024,66.2,°F,53.8,°F,43.8,°F,57.3,°F,50.3,...,0.1,mph,0.0,mph,30.03,in,29.91,in,0.05,in
4,1/4/2024,64.0,°F,58.1,°F,55.5,°F,60.2,°F,57.1,...,0.7,mph,0.0,mph,29.94,in,29.76,in,0.38,in
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,10/12/2024,70.0,°F,68.0,°F,64.9,°F,69.6,°F,67.6,...,2.8,mph,0.0,mph,30.12,in,29.99,in,0.27,in
1496,10/13/2024,76.6,°F,67.5,°F,62.2,°F,72.0,°F,66.0,...,2.3,mph,0.0,mph,30.24,in,30.09,in,1.96,in
1497,10/14/2024,80.1,°F,68.1,°F,62.2,°F,73.4,°F,66.4,...,0.8,mph,0.0,mph,30.24,in,30.06,in,0.00,in
1498,10/15/2024,69.8,°F,66.5,°F,64.4,°F,69.4,°F,66.1,...,3.5,mph,0.0,mph,30.09,in,29.87,in,0.68,in


In [37]:
df.to_csv("datos/tabla_a_limpiar.csv")
df = pd.read_csv("datos/tabla_a_limpiar.csv",index_col="Unnamed: 0")

In [38]:
df = df.drop(columns=["2","4","6","8","10","12","14","16","18","20","22","24","26","28","30"])
columnas = ["Date","Temperature High (Fº)", "Temperature Avg (Fº)","Temperature Low (Fº)","Dew Point High (Fº)",
            "Dew Point Avg (Fº)","Dew Point Low (Fº)","Humidity High (%)","Humidity Avg (%)","Humidity Low (%)",
            "Speed High (mph)","Speed Avg (mph)","Speed Low (mph)","Pressure High (in)","Pressure Low (in)","Precipitation Accumulation Sum (in)"]
df.rename(columns={"0":columnas[0],"1":columnas[1],"3":columnas[2],"5":columnas[3],
                   "7":columnas[4],"9":columnas[5],"11":columnas[6],"13":columnas[7],
                   "15":columnas[8],"17":columnas[9],"19":columnas[10],"21":columnas[11],
                   "23":columnas[12],"25":columnas[13],"27":columnas[14],"29":columnas[15]},inplace=True)
df["Date"] = pd.to_datetime(df["Date"],format="mixed")
df["Humidity High (%)"] = df["Humidity High (%)"].astype(int)
df["Humidity Avg (%)"] = df["Humidity Avg (%)"].astype(int)
df["Humidity Low (%)"] = df["Humidity Low (%)"].astype(int)

DateParseError: Unknown datetime string format, unable to parse: Date, at position 0